In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict

In [2]:
class State(TypedDict):
    runs:int
    balls:int
    four:int
    six:int

    sr:float
    bpb:float
    boundary_percentage:float
    summary:str

In [15]:
def calculate_sr(state:State)->State:
    state["sr"] = (state["runs"]/state["balls"])*100
    return {"sr":state["sr"]}

def calculate_bpb(state:State)->State:
    state["bpb"] = (state["balls"]/(state["four"] + state["six"]))
    return {"bpb": state["bpb"]}

def calculate_boundary_percentage(state:State)->State:
    state["boundary_percentage"] = (((state['four'] * 4) + (state['six'] * 6))/state['runs'])*100
    return {"boundary_percentage": state["boundary_percentage"]}

def summary(state: State)->State:

    state["summary"] = f"""
Strike Rate - {state['sr']} \n
Balls per boundary - {state['bpb']} \n
Boundary percent - {state['boundary_percentage']}
"""
    
    return {"summary":state["summary"]}

In [16]:
graph = StateGraph(State)

graph.add_node("calculate_sr", calculate_sr),
graph.add_node("calculate_bpb", calculate_bpb)
graph.add_node("calculate_boundary_percentage", calculate_boundary_percentage)
graph.add_node("summary", summary)

graph.add_edge(START, "calculate_sr")
graph.add_edge(START, "calculate_bpb")
graph.add_edge(START, "calculate_boundary_percentage")

graph.add_edge("calculate_sr", "summary")
graph.add_edge("calculate_bpb", "summary")
graph.add_edge("calculate_boundary_percentage", "summary")

graph.add_edge("summary", END)

workflow = graph.compile()

In [17]:
intial_state = {
    'runs': 100,
    'balls': 50,
    'four': 6,
    'six': 4
}

workflow.invoke(intial_state)

{'runs': 100,
 'balls': 50,
 'four': 6,
 'six': 4,
 'sr': 200.0,
 'bpb': 5.0,
 'boundary_percentage': 48.0,
 'summary': '\nStrike Rate - 200.0 \n\nBalls per boundary - 5.0 \n\nBoundary percent - 48.0\n'}